In [ ]:
JIRA_EMAIL = "xxxxxxxxxxxxx.xxxxxxxxxxxxx@xxxxxxxxxx.com"
JIRA_TOKEN = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
JIRA_URL = "https://xxxxxxxxxxxxxx.atlassian.net"

In [246]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def get_filtered_jira_issues(df_tickets):
    """
    Récupère les tickets Jira correspondant aux composants, types et champs personnalisés d'un DataFrame :
    - dont "fields.resolution.name" == "Done"
    - dont "fields.components" contient au moins un composant présent dans df_tickets
    - dont "fields.customfield_10116.value" contient une valeur présente dans df_tickets (filtré après)
    - dont "fields.issuetype.name" correspond à un type présent dans df_tickets
    - créés dans l'année précédente par rapport à leur propre date de création

    Args:
        df_tickets (pd.DataFrame): Un DataFrame avec des colonnes Jira standards

    Returns:
        pd.DataFrame: Tickets filtrés
    """
    if "key" not in df_tickets.columns:
        raise ValueError("Le DataFrame doit contenir une colonne 'key' avec les identifiants Jira")

    auth = (JIRA_EMAIL, JIRA_TOKEN)
    headers = {"Accept": "application/json"}
    results = []

    # Extraction des composants, types d'issue et valeurs custom à réutiliser
    component_set = set()
    issuetype_set = set()
    custom_value_set = set()

    if "fields.components" in df_tickets.columns:
        df_tickets["fields.components"].dropna().apply(
            lambda lst: [component_set.add(comp["name"]) for comp in lst if isinstance(comp, dict)]
        )
    if "fields.issuetype.name" in df_tickets.columns:
        issuetype_set = set(df_tickets["fields.issuetype.name"].dropna().tolist())
    if "fields.customfield_10116.value" in df_tickets.columns:
        custom_value_set = set(df_tickets["fields.customfield_10116.value"].dropna().tolist())

    url = f"{JIRA_URL}/rest/api/latest/search"

    for issuetype in issuetype_set:
        for component in component_set:
            jql = f"created >= -365d AND component = \"{component}\" AND issuetype = \"{issuetype}\" AND resolution = Done"
            start_at = 0

            while True:
                params = {
                    "jql": jql,
                    "startAt": start_at,
                    "maxResults": 100
                }

                response = requests.get(url, auth=auth, headers=headers, params=params)

                if response.status_code != 200:
                    print(f"⚠️ Erreur API Jira : {response.status_code} {response.text}")
                    break

                data = response.json()
                issues = data.get("issues", [])
                total = data.get("total", 0)

                for issue in issues:
                    fields = issue.get("fields", {})
                    resolution = fields.get("resolution", {}).get("name", "")
                    components = fields.get("components", [])
                    custom_value = fields.get("customfield_10116", {}).get("value", None)
                    issuetype_value = fields.get("issuetype", {}).get("name", None)
                    created = fields.get("created", "")

                    if not created:
                        continue

                    created_date = datetime.strptime(created[:10], "%Y-%m-%d")
                    date_min = created_date - timedelta(days=365)

                    component_names = [comp.get("name") for comp in components if isinstance(comp, dict)]
                    has_common_component = bool(component_set.intersection(component_names))
                    has_common_custom_value = custom_value in custom_value_set
                    has_common_issuetype = issuetype_value in issuetype_set

                    if (resolution == "Done" and
                        has_common_component and
                        has_common_issuetype and
                        date_min <= created_date <= created_date):

                        flat = pd.json_normalize(issue)
                        if has_common_custom_value:
                            results.append(flat)

                start_at += len(issues)
                if start_at >= total:
                    break

    if results:
        return pd.concat(results, ignore_index=True)
    return pd.DataFrame()


In [247]:
import requests
import pandas as pd

auth = (JIRA_EMAIL, JIRA_TOKEN)
headers = {"Accept": "application/json"}

# Ticket à récupérer
ticket_id = "CMH-15279"
url = f"{JIRA_URL}/rest/api/latest/issue/{ticket_id}"

# Appel API
response = requests.get(url, auth=auth, headers=headers)
response.raise_for_status()

# Transformation en DataFrame
issue_data = response.json()
df_ticket = pd.json_normalize(issue_data)
print(df_ticket.head())


                                              expand       id  \
0  renderedFields,names,schema,operations,editmet...  7974132   

                                                self        key  \
0  https://arvato-scs.atlassian.net/rest/api/late...  CMH-15279   

  fields.parent.id fields.parent.key  \
0          2626824         CMH-10426   

                                  fields.parent.self  \
0  https://arvato-scs.atlassian.net/rest/api/2/is...   

  fields.parent.fields.summary  \
0             431 BMS - France   

                    fields.parent.fields.status.self  \
0  https://arvato-scs.atlassian.net/rest/api/2/st...   

  fields.parent.fields.status.description  ... fields.customfield_10080  \
0                                          ...                     None   

  fields.customfield_10081 fields.customfield_10082 fields.customfield_12382  \
0                     None                     None                     None   

   fields.customfield_10087 fields.customfield

In [248]:
# # Ajoute la colonne 'Matricule' si elle n'existe pas dans df_ticket
# if "Matricule" not in df_ticket.columns:
#     matricule = "900265"  # ou récupéré dynamiquement d'un formulaire ou input utilisateur
#     df_ticket["Matricule"] = matricule

# Appel API pour tickets similaires
df_all_ticket = get_filtered_jira_issues(df_ticket)

# Fusion des tickets initiaux et similaires
df_all_ticket = pd.concat([df_ticket, df_all_ticket], ignore_index=True)

# Suppression des doublons par clé unique Jira
df_all_ticket = df_all_ticket.drop_duplicates(subset="key").reset_index(drop=True)


C:\Users\atdf2\AppData\Local\Temp\ipykernel_21772\3261151567.py:98: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(results, ignore_index=True)
C:\Users\atdf2\AppData\Local\Temp\ipykernel_21772\3602510204.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all_ticket = pd.concat([df_ticket, df_all_ticket], ignore_index=True)


In [ ]:
import sqlite3

def lire_zh12_depuis_sqlite(sqlite_path, table_name, jira_keys):
    """
    Charge uniquement les lignes de la base SQLite dont la colonne Jira est dans jira_keys.
    """
    conn = sqlite3.connect(sqlite_path)

    # Convertir les clés Jira en tuple SQL-safe pour l'IN clause
    placeholders = ','.join(['?'] * len(jira_keys))
    query = f"SELECT * FROM {table_name} WHERE Jira IN ({placeholders})"

    zh12 = pd.read_sql_query(query, conn, params=tuple(jira_keys))
    conn.close()
    return zh12

def tester_requete_sql(sqlite_path: str, table: str, tickets):
    """
    Exécute une requête SELECT * sur une table SQLite filtrée par une ou plusieurs clés Jira.
    
    Args:
        sqlite_path (str): Chemin vers la base SQLite
        table (str): Nom de la table (ex: "zh12")
        tickets (str | list[str]): Ticket Jira ou liste de tickets
        max_lignes (int): Nombre max de lignes à afficher
    """
    try:
        conn = sqlite3.connect(sqlite_path)
        print(f"📡 Connexion ouverte vers {sqlite_path}")

        # Préparation des tickets
        if isinstance(tickets, str):
            tickets = [tickets]
        tickets = [str(t).strip() for t in tickets if t]

        # Clause WHERE Jira IN (?, ?, ...)
        placeholders = ','.join(['?'] * len(tickets))
        requete_sql = f"SELECT * FROM {table} WHERE Jira IN ({placeholders})"

        print("🔎 Requête préparée :")
        print(f"{requete_sql}")
        print("🔑 Paramètres :", tickets)

        df = pd.read_sql_query(requete_sql, conn, params=tuple(tickets))
        conn.close()

        print(f"✅ {len(df)} ligne(s) récupérée(s).")
        return df

    except Exception as e:
        print(f"❌ Erreur : {e}")
        return pd.DataFrame()


def construire_type_etendu(row, colonnes):
    valeurs = []
    for col in colonnes:
        val = str(row.get(col, "")).strip()
        if val:
            valeurs.append(val)
    return "__".join(valeurs) if valeurs else None

def calculer_historique_1an(df, df_reference, type_col="Type_Étendu", date_col="fields.created", projet_col="fields.project.key", duree_col="Duree"):
    history = []
    for idx, row in df.iterrows():
        typ = row[type_col]
        date = row[date_col]
        projet = row[projet_col]

        if pd.isna(typ) or pd.isna(date) or pd.isna(projet):
            history.append(0)
            continue

        date_min = date - timedelta(days=365)
        past = df_reference[
            (df_reference[projet_col] == projet) &
            (df_reference[date_col] < date) &
            (df_reference[date_col] >= date_min) &
            (df_reference[type_col] == typ)
        ]
        print(f"🔍 Historique pour {typ} dans {projet} entre {date_min} et {date}: {len(past)} entrées trouvées")
        mean_val = past[duree_col].mean() if not past.empty else 0
        history.append(mean_val)

    return history

# --------- MAIN PIPELINE ---------

def traiter_historique_1an(df_ticket: pd.DataFrame, df_merged: pd.DataFrame, zh12_path: str, date_commencement: str, matricule: str):
    """
    Calcule l'historique glissant 1 an pour les tickets Jira (df_ticket) enrichis via df_merged.
    Ajoute les tickets manquants dans ZH12 avec durée = 0.
    """
    # Copie des tickets enrichis
    jira_df = df_merged.copy()

    # Charger ZH12 uniquement pour les clés présentes dans Jira
    keep_keys = list(set(jira_df["key"]))
    zh12 = lire_zh12_depuis_sqlite(zh12_path, "zh12", keep_keys)
    print(f"📥 Chargement de {len(zh12)} lignes de ZH12 pour {len(keep_keys)} tickets Jira")

    # Agrégation ZH12 si non vide
    if not zh12.empty:
        zh12["Durée tâche (heures)"] = pd.to_numeric(zh12["Durée tâche (heures)"], errors="coerce")
        zh12["Date"] = pd.to_datetime(zh12["Date"], errors="coerce")
        aggr = zh12.groupby(["Matricule", "Jira"], as_index=False).agg({
            "Durée tâche (heures)": "sum",
            "Code Service": "first",
            "Date": "min"
        }).rename(columns={"Date": "Date commence"})
    else:
        aggr = pd.DataFrame(columns=["Matricule", "Jira", "Durée tâche (heures)", "Code Service", "Date commence"])

    # Ajouter les tickets manquants (non déjà dans aggr)
    missing_keys = [k for k in df_ticket["key"] if k not in aggr["Jira"].values]

    if missing_keys:
        rows_to_add = pd.DataFrame({
            "Matricule": [matricule] * len(missing_keys),
            "Jira": missing_keys,
            "Durée tâche (heures)": [0] * len(missing_keys),
            "Code Service": [None] * len(missing_keys),
            "Date commence": [date_commencement] * len(missing_keys)
        })
        aggr = pd.concat([aggr, rows_to_add], ignore_index=True)
    print(aggr.head())

    # Merge avec les données Jira
    merged_df = aggr.merge(jira_df, left_on="Jira", right_on="key", how="left")
    print(f"🔗 Fusion des données Jira et ZH12 : {len(merged_df)} lignes après fusion")

    # Nettoyage des composants
    merged_df["fields.components"] = merged_df["fields.components"].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith("[") else x
    )
    merged_df["components"] = merged_df["fields.components"].apply(
        lambda comps: " / ".join(
            [c["name"] for c in comps if isinstance(c, dict) and "name" in c]
        ) if isinstance(comps, list) else None
    )

    # Construction du type étendu
    colonnes_type = ["fields.issuetype.name", "components", "fields.customfield_10116.value"]
    merged_df["Type_Étendu"] = merged_df.apply(lambda r: construire_type_etendu(r, colonnes_type), axis=1)
    merged_df = merged_df.dropna(subset=["Type_Étendu"])
    merged_df["Type_Étendu"] = merged_df["Type_Étendu"].astype(str)
    merged_df = merged_df[merged_df["Type_Étendu"].str.strip() != ""].reset_index(drop=True)

    # Nettoyage des dates
    merged_df["fields.created"] = pd.to_datetime(merged_df["fields.created"], errors="coerce", utc=True).dt.tz_convert(None)

    print(merged_df[["Type_Étendu", "fields.created"]])

    # Appliquer Date commence pour les lignes de df_ticket
    date_val = pd.to_datetime(date_commencement)
    keys_ticket = set(df_ticket["key"])
    mask = merged_df["key"].isin(keys_ticket)
    merged_df.loc[mask, "Date commence"] = date_val
    merged_df["Date commence"] = pd.to_datetime(merged_df["Date commence"], errors="coerce", utc=True).dt.tz_convert(None)

    # Colonnes temporelles
    merged_df["annee_creation"] = merged_df["fields.created"].dt.year
    merged_df["mois_creation"] = merged_df["fields.created"].dt.month
    merged_df["jour_semaine"] = merged_df["fields.created"].dt.weekday
    merged_df["delai_creation_action_h"] = (
        (merged_df["Date commence"] - merged_df["fields.created"]).dt.total_seconds() / 3600
    )

    merged_df.rename(columns={"Durée tâche (heures)": "Duree"}, inplace=True)

    # Calcul Historique uniquement sur les tickets initiaux
    merged_df["Historique_1an"] = 0
    # subset_to_compute = merged_df[mask].copy()
    merged_df[mask]["Historique_1an"] = calculer_historique_1an(merged_df[mask], merged_df)

    return merged_df[mask]


In [250]:
result = traiter_historique_1an(
    df_ticket=df_ticket,
    df_merged=df_all_ticket,
    zh12_path="zh12_v2.sqlite",
    date_commencement="2025-07-13",
    matricule="900265"
)


📥 Chargement de 1 lignes de ZH12 pour 10 tickets Jira
  Matricule       Jira  Durée tâche (heures)  Code Service Date commence
0   7000755  CMH-15279                   1.0  SCM-IT-FR-HC    2025-06-04
🔗 Fusion des données Jira et ZH12 : 1 lignes après fusion
                         Type_Étendu          fields.created
0  Change__JP6 - 431 BMS__Healthcare 2025-06-03 12:22:56.417
🔍 Historique pour Change__JP6 - 431 BMS__Healthcare dans CMH entre 2024-06-03 12:22:56.417000 et 2025-06-03 12:22:56.417000: 0 entrées trouvées


In [251]:
result

,Matricule,Jira,Duree,Code Service,Date commence,expand,id,self,key,fields.parent.id,...,fields.customfield_10126.value,fields.customfield_10126.id,fields.customfield_10104,components,Type_Étendu,annee_creation,mois_creation,jour_semaine,delai_creation_action_h,Historique_1an
0,7000755,CMH-15279,1.0,SCM-IT-FR-HC,2025-07-13,"renderedFields,names,schema,operations,editmet...",7974132,https://arvato-scs.atlassian.net/rest/api/late...,CMH-15279,2626824,...,NaN,NaN,NaN,JP6 - 431 BMS,Change__JP6 - 431 BMS__Healthcare,2025,6,1,947.617662,0


In [252]:
# import pandas as pd
# import sqlite3

# import pandas as pd
# import sqlite3

# def convertir_type_sql(pandas_dtype):
#     """
#     Convertit un type pandas (dtype) en type SQL standard.
#     """
#     if pd.api.types.is_integer_dtype(pandas_dtype):
#         return "INTEGER"
#     elif pd.api.types.is_float_dtype(pandas_dtype):
#         return "REAL"
#     else:
#         return "TEXT"

# def importer_excel_dans_sqlite(excel_path, sqlite_path, table_name="zh12"):
#     # Lecture du fichier Excel avec inférence des types
#     df = pd.read_excel(excel_path)
#     df = df.dropna(subset=["Matricule", "Date", "# de tâche"])

#     # Inférer les types de chaque colonne pour SQL
#     schema_sql = []
#     for col in df.columns:
#         sql_type = convertir_type_sql(df[col].dtype)
#         schema_sql.append(f"[{col}] {sql_type}")

#     # Ajout de la clé primaire composite
#     schema_sql.append("PRIMARY KEY (Matricule, Date, [# de tâche])")
#     create_stmt = f"CREATE TABLE IF NOT EXISTS {table_name} (\n  " + ",\n  ".join(schema_sql) + "\n)"

#     # Connexion SQLite
#     conn = sqlite3.connect(sqlite_path)
#     cursor = conn.cursor()
#     cursor.execute(create_stmt)
#     conn.commit()

#     # Supprimer les doublons existants
#     for _, row in df.iterrows():
#         cursor.execute(f"""
#             DELETE FROM {table_name}
#             WHERE Matricule = ? AND Date = ? AND [# de tâche] = ?
#         """, (str(row["Matricule"]), str(row["Date"]), str(row["# de tâche"])))

#     # Réinsertion
#     df.to_sql(table_name, conn, if_exists="append", index=False)
#     print(f"✅ {len(df)} lignes insérées dans '{table_name}' (avec remplacement des doublons).")

#     conn.commit()
#     conn.close()


# importer_excel_dans_sqlite("export_ZH12.xlsx", "zh12_v2.sqlite")


In [2]:
import sqlite3
import pandas as pd

def tester_requete_sql(sqlite_path: str, table: str, tickets, max_lignes: int = 10):
    """
    Exécute une requête SELECT * sur une table SQLite filtrée par une ou plusieurs clés Jira.
    
    Args:
        sqlite_path (str): Chemin vers la base SQLite
        table (str): Nom de la table (ex: "zh12")
        tickets (str | list[str]): Ticket Jira ou liste de tickets
        max_lignes (int): Nombre max de lignes à afficher
    """
    try:
        conn = sqlite3.connect(sqlite_path)
        print(f"📡 Connexion ouverte vers {sqlite_path}")

        # Préparation des tickets
        if isinstance(tickets, str):
            tickets = [tickets]
        tickets = [str(t).strip() for t in tickets if t]

        # Clause WHERE Jira IN (?, ?, ...)
        placeholders = ','.join(['?'] * len(tickets))
        requete_sql = f"SELECT * FROM {table} WHERE Jira IN ({placeholders}) LIMIT {max_lignes}"

        print("🔎 Requête préparée :")
        print(f"{requete_sql}")
        print("🔑 Paramètres :", tickets)

        df = pd.read_sql_query(requete_sql, conn, params=tuple(tickets))
        conn.close()

        print(f"✅ {len(df)} ligne(s) récupérée(s).")
        return df

    except Exception as e:
        print(f"❌ Erreur : {e}")
        return pd.DataFrame()


In [3]:
# Un seul ticket
# df_test = tester_requete_sql("zh12_v2.sqlite", "zh12", "CMH-15279")

# # Ou plusieurs tickets
df_test = tester_requete_sql("zh12_v2.sqlite", "zh12", ["CMH-15279", "CMH-13626"])

df_test

📡 Connexion ouverte vers zh12_v2.sqlite
🔎 Requête préparée :
SELECT * FROM zh12 WHERE Jira IN (?,?) LIMIT 10
🔑 Paramètres : ['CMH-15279', 'CMH-13626']
✅ 1 ligne(s) récupérée(s).


,Matricule,Date,# de tâche,Code tâche,Jira,Code Service,Heures déclarées,Durée tâche (heures),Unité quantité base,Créé par,Saisi le,Heure de création,Confirmé,Commentaire,Durée tâche (heures).1,Unité quantité base.1
0,7000755,2025-06-04 00:00:00,2,PCBMS,CMH-15279,SCM-IT-FR-HC,01:00:00.000000,1.0,H,NGUY169,2025-06-30 00:00:00,18:08:22.000000,X,None,0,None
